# GPU
查看显卡信息

In [1]:
!nvidia-smi

Mon Sep 19 21:21:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.59       Driver Version: 516.59       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8    N/A /  N/A |    660MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

计算设备

In [2]:
import torch 
from torch import nn
torch.device('cpu'),torch.cuda.device('cuda'),torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x264f79f1af0>)

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [6]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在的设备

In [7]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

储存在GPU上面

In [8]:
X = torch.ones(2,3,device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

第二个GPU上创建一个随机张量

In [12]:
# 只有一个GPU，用cpu 和 gpu 模拟了 
Y = torch.rand(2, 3, device=try_gpu(1))
Y.device

device(type='cpu')

In [14]:
Z = X.cpu()
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]])


现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加

In [15]:
Y + Z

tensor([[1.6273, 1.2483, 1.6040],
        [1.6630, 1.6099, 1.5026]])

神经网络与GPU

In [16]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)


tensor([[0.6145],
        [0.6145]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)